# SQL

### Introducción

Durante esta actividad vamos a aprender los conceptos básicos de SQL. Vamos a aprender a insertar, eliminar y actualizar datos de una base de datos además de hacer consultas sencillas sobre ellos.

### Requisitos

Para esta actividad, así como en las siguientes actividades de SQL vamos a utilizar

- Python 3
- Jupyter
- La librería `ipython-sql`. Para instalarla pueden hacerlo con `pip3`:
```
pip3 install ipython-sql
```
- SQLite3, que viene junto con la instalación de Python

### Outline

En esta actividad aprendermos a:

- Crear y modificar tablas.
- Insertar, eliminar y actualizar datos.
- Crear llaves en las tablas.
- Hacer consultas que involucren selección, proyección, producto cruz y operaciones de conjuntos.
- Consultas básicas de agregación.

### Esquema

Para esta actividad vamos a trabajar con el siguiente esquema:

- `Capitanes(cid INT PRIMARY KEY, cnombre VARCHAR(100), crating FLOAT, cedad INT)`
- `Botes(bid INT PRIMARY KEY, bnombre VARCHAR(100), bcolor VARCHAR(100))`
- `Reservas(cid INT, bid INT, fecha DATE, PRIMARY KEY(cid, bid))`

Que corresponde a capitanes que reservan ciertos botes. El `cid` y `bid` en la tabla `Reservas` proviene de las tablas `Capitanes` y `Botes` respectivamente.

## Actividad

### Crear tablas, eliminar tablas y llaves primarias

En esta actividad vamos a utilizar `ipython-sql` para tener acceso a SQL desde este _notebook_. Lo vamos a importar a continuación.

In [3]:
%load_ext sql

Ahora vamos a crear una nueva base de datos para esta actividad. Esta base de datos se llamará `capitanes.db`.

In [4]:
%sql sqlite:///capitanes.db

**Ojo**: cada vez que anteponemos `%sql` es porque esa línea corresponde a un comando a SQL que va a la base de datos en la que estamos trabajando (en este caso `capitanes.db`). Si queremos tener todo un bloque con instrucciones SQL tenemos que usar `%%sql` (habrán ejemplos de esto a lo largo del _notebook_).

Tu primera prueba será crear una base de datos de acuerdo el esquema de arriba. La sintaxis para crear tablas en SQL es la siguiente (si están trabajando desde el cliente, después de cada sentencia SQL debes poner un punto y coma `;`):

```SQL
CREATE TABLE <Nombre Tabla> (<atributo_1> tipo, ... , <attributo_N> tipo)
```

Por ejemplo, para crear la tabla de capitanes:

In [5]:
%%sql
DROP TABLE IF EXISTS Capitanes;
CREATE TABLE Capitanes(cid INT, cnombre VARCHAR(100), crating FLOAT, cedad INT);

 * sqlite:///capitanes.db
Done.
Done.


[]

En la celda anterior estamos eliminando la tabla si es que existe para luego crear una tabla según lo requerido por el esquema. Pero cuidado! Olvidamos agregar la llave primaria, por lo que vamos a **eliminar** la tabla y crearla de nuevo. Además, vamos a agregar un valor por defecto al atributo `crating`, que en este caso será 0.

In [6]:
%%sql
DROP TABLE Capitanes;
CREATE TABLE Capitanes(cid INT PRIMARY KEY, cnombre VARCHAR(100), crating FLOAT DEFAULT 0, cedad INT);

 * sqlite:///capitanes.db
Done.
Done.


[]

Otra opción es agregar la llave primaria al final de la instrucción `CREATE TABLE`. Para ver un ejemplo, vamos a crear la tabla de reservas.

In [7]:
%%sql
DROP TABLE IF EXISTS Reservas;
CREATE TABLE Reservas(cid INT, bid INT, fecha DATE, PRIMARY KEY(cid, bid));

 * sqlite:///capitanes.db
Done.
Done.


[]

En el caso anterior, tenemos una llave primaria compuesta. En general, para crear una tabla con llave primaria, lo señalamos al final, de la forma:

```SQL
CREATE TABLE <Nombre Tabla> (
    <atributo_1> tipo, ...,
    <attributo_N> tipo,
    PRIMARY KEY(<atributos separados por coma (,)>))
```

**1.** Ahora es tu turno. En la celda a continuación debes crear la tabla de botes. No olvides agregar su llave primaria.

In [8]:
%%sql
CREATE TABLE Botes (
    bid INT PRIMARY KEY,
    bnombre VARCHAR(100),
    bcolor VARCHAR(100)
);

 * sqlite:///capitanes.db
Done.


[]

### Tipos de datos

Al crear las tablas disponemos varios tipos de datos. Por ahora destacamos los siguientes:

- Caracteres (_Strings_):
  - `CHAR(20)`: _Strings_ de largo fijo.
  - `VARCHAR(20)`: _Strings_ de largo variable.
  
- Números:
  - `INT`
  - `FLOAT`
  - `SMALLINT`
  
- Tiempos y fechas:
  - `DATE`: fecha.
  - `TIME`: hora.
  - `TIMESTAMP`: fecha y hora.

### Insertar, eliminar y modificar elementos en tablas

Si ejecutamos la consulta `SELECT * FROM Capitanes` notaremos que el resultado es vacío. Esto es evidente, ya que esta consulta me retorna todo lo que tengo en la tabla `Capitanes` y actualmente no hemos insertado nada.

In [9]:
%sql SELECT * FROM Capitanes;

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad


Para insertar valores, la forma básica es la siguiente:

```SQL
INSERT INTO <Nombre Tabla>
VALUES (<valor atributo 1> , ..., <valor atributo N>)
```

Por ejemplo si ejecutamos la consulta:

```SQL
INSERT INTO Capitanes
VALUES(1, 'Claudio', 0, 35)
```

estamos insertando un Capitán con `cid` 1, `cnombre` Claudio, `crating` 0 y `cedad` 35.

In [10]:
%sql INSERT INTO Capitanes VALUES(1, 'Claudio', 0, 35)

 * sqlite:///capitanes.db
1 rows affected.


[]

Recordemos que habíamos señalado que la llave primaria de la tabla `Capitanes` era el `cid`. Veamos que pasa si intentamos insertar un capitán con el mismo `cid`.

In [11]:
%sql INSERT INTO Capitanes VALUES(1, 'Claudio Bravo', 0, 35)

 * sqlite:///capitanes.db
(sqlite3.IntegrityError) UNIQUE constraint failed: Capitanes.cid
[SQL: INSERT INTO Capitanes VALUES(1, 'Claudio Bravo' , 0, 35)]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


La consulta anterior debería haber arrojado un error. Así que vamos a cambiar el `cid`. Sin embargo, vamos a omitir algunos atributos.

In [12]:
%sql INSERT INTO Capitanes(cid, cnombre) VALUES(13, 'Claudio Bravo')

 * sqlite:///capitanes.db
1 rows affected.


[]

In [13]:
%sql SELECT * FROM Capitanes

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
1,Claudio,0.0,35
13,Claudio Bravo,0.0,None


Si te das cuenta, ahora tenemos dos tuplas en la tabla de capitanes. Como podrías haber esperado, dado que omitimos el valor para ciertos atributos en la segunda inserción, el `crating` tomó el valor _default_. Pero ojo! el atributo `cedad` tomó un valor **nulo**. Más adelante en el curso vamos a ahondar en esta temática y ver por qué los valores nulos pueden ser un problema. Ahora vamos a agregar una nueva tupla para hacer algunas consultas sencillas.

In [14]:
%sql INSERT INTO Capitanes VALUES(23, 'Arturo Vidal', 8, 31)

 * sqlite:///capitanes.db
1 rows affected.


[]

In [15]:
%sql SELECT * FROM Capitanes;

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
1,Claudio,0.0,35
13,Claudio Bravo,0.0,None
23,Arturo Vidal,8.0,31


In [16]:
%sql SELECT * FROM Capitanes WHERE cnombre='Claudio'

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
1,Claudio,0.0,35


En la consulta anterior estamos filtrando por Capitanes donde su nombre sea `'Claudio'`. Si queremos que el nombre **contenga** `'Claudio'` usamos la instrucción `LIKE`:

In [17]:
%sql SELECT * FROM Capitanes WHERE cnombre LIKE '%Claudio%'

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
1,Claudio,0.0,35
13,Claudio Bravo,0.0,None


Pueden notar que hemos añadido un `%` al _string_. Esto es porque antes y después de `Claudio` queremos permitir cualquier secuencia de caracteres. Para el `LIKE` tenemos dos instrucciones posibles:

- `%` que significa cualquier secuencia de caracteres.
- `_` que significa un caracter, pero **solamente** uno.

Como no estamos seguros de mantener a `Claudio` en la tabla `Capitanes` lo vamos a eliminar. Para eliminar tuplas lo hacemos con la instrucción `DELETE`. Veamos que pasa al ejecutar la siguiente consulta:

In [18]:
%%sql
DELETE FROM Capitanes
WHERE cnombre LIKE '%Claudio%'

 * sqlite:///capitanes.db
2 rows affected.


[]

In [19]:
%sql SELECT * FROM Capitanes

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
23,Arturo Vidal,8.0,31


Lo que hicimos fue borrar de la tabla `Capitanes` todas las tuplas que satisfacen la condición en el `WHERE` de la consulta. Para actualizar una tupla usamos la instrucción `UPDATE`, como veremos a continuación:

In [20]:
%sql UPDATE Capitanes SET cnombre='King Arturo' WHERE cid=23

 * sqlite:///capitanes.db
1 rows affected.


[]

In [21]:
%sql SELECT * FROM Capitanes

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
23,King Arturo,8.0,31


### Modificando tablas

En SQL es posible modificar tablas. Por ejemplo si quisieramos eliminar el atributo `crating` podríamos ejecutar la consulta:

```SQL
ALTER TABLE Capitanes DROP COLUMN crating
```

O agregar una columna, como por ejemplo:

```SQL
ALTER TABLE Capitanes ADD COLUMN rut VARCHAR(20)
```

### Consultas básicas en SQL

Para comenzar a hacer consultas primero debemos llenar nuestras tablas. Asegurate que las instancias de las tablas sean las siguientes:

#### Capitanes

| cid | cnombre     | crating | cedad |
|-----|-------------|---------|-------|
| 23  | King Arturo | 8       | 31    |
| 29  | Juan        | 1       | 33    |
| 31  | Andy        | 8       | 55    |
| 32  | Felipe      | 8.4     | 25    |
| 58  | Oscar       | 10      | 35    |
| 64  | Isidora     | 7.5     | 35    |
| 71  | Pedro       | 10      | 16    |
| 74  | Isidora     | 9       | 35    |
| 85  | Rosa        | 3       | 25    |
| 95  | Romano      | 5.5     | 63    |

#### Botes

| bid | bnombre   | bcolor |
|-----|-----------|--------|
| 101 | Catamaran | Azul   |
| 102 | Catamaran | Rojo   |
| 103 | Endurance | Verde  |
| 104 | Yate      | Rojo   |

#### Reservas

| cid | bid | fecha    |
|-----|-----|----------|
| 23  | 101 | 10/10/16 |
| 23  | 102 | 10/10/16 |
| 23  | 103 | 8/10/16  |
| 23  | 104 | 7/10/17  |
| 31  | 102 | 10/11/17 |
| 31  | 103 | 6/11/18  |
| 31  | 104 | 12/11/18 |
| 64  | 101 | 5/9/18   |
| 64  | 102 | 8/9/18   |
| 74  | 103 | 8/9/18   |

**Hint**: Para insertar un tipo fecha en SQLITE tienes que ingresarlo entre comillas simples en el formato `'YYYY-MM-DD'`, por ejemplo:

In [22]:
%sql INSERT INTO Reservas VALUES(23, 101, '2018-10-10')

 * sqlite:///capitanes.db
1 rows affected.


[]

In [23]:
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (23, 101, '2016-10-10');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (23, 102, '2016-10-10');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (23, 103, '2016-08-10');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (23, 104, '2017-07-10');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (31, 102, '2017-10-11');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (31, 103, '2018-11-06');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (31, 104, '2018-11-12');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (64, 101, '2018-09-05');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (64, 102, '2018-09-08');
%sql INSERT INTO Reservas (cid, bid, fecha) VALUES (74, 103, '2018-09-08');

 * sqlite:///capitanes.db
(sqlite3.IntegrityError) UNIQUE constraint failed: Reservas.cid, Reservas.bid
[SQL: INSERT INTO Reservas (cid, bid, fecha) VALUES (23, 101, '2016-10-10' );]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.


[]

In [24]:
%sql INSERT INTO Botes (bid, bnombre, bcolor) VALUES (101, 'Catamaran', 'Azul');
%sql INSERT INTO Botes (bid, bnombre, bcolor) VALUES (102, 'Catamaran', 'Rojo');
%sql INSERT INTO Botes (bid, bnombre, bcolor) VALUES (103, 'Endurance', 'Verde');
%sql INSERT INTO Botes (bid, bnombre, bcolor) VALUES (104, 'Yate', 'Rojo');

 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.


[]

In [25]:
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (23, 'King Arturo', 8, 31);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (29, 'Juan', 1, 33);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (31, 'Andy', 8, 55);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (32, 'Felipe', 8.4, 25);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (58, 'Oscar', 10, 35);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (64, 'Isidora', 7.5, 35);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (71, 'Pedro', 10, 16);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (74, 'Isidora', 9, 35);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (85, 'Rosa', 3, 25);
%sql INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (95, 'Romano', 5.5, 63);


 * sqlite:///capitanes.db
(sqlite3.IntegrityError) UNIQUE constraint failed: Capitanes.cid
[SQL: INSERT INTO Capitanes (cid, cnombre, crating, cedad) VALUES (23, 'King Arturo' , 8, 31);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.
 * sqlite:///capitanes.db
1 rows affected.


[]

Las consultas más básicas son de la forma `SELECT - FROM - WHERE`. En general, la consulta de álgebra relacional:

$$
\pi_{a_1, \dots, a_n}(\sigma_{\text{condiciones}}(R_1 \times R_m))
$$

se traduce en SQL como:

```SQL
SELECT a_1, ..., a_n
FROM R_1, ..., R_m
WHERE <condiciones>
```

**2.** Prueba la siguiente consulta:

In [26]:
%sql SELECT cid, cnombre, crating, cedad FROM Capitanes

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
23,King Arturo,8.0,31
29,Juan,1.0,33
31,Andy,8.0,55
32,Felipe,8.4,25
58,Oscar,10.0,35
64,Isidora,7.5,35
71,Pedro,10.0,16
74,Isidora,9.0,35
85,Rosa,3.0,25
95,Romano,5.5,63


¿Qué es lo que debería hacer la consulta anterior? Recuerda que si lo que quieres es proyectar todos los atributos puedes usar `*`.

**3.** Ahora realiza dos consultas, una para obtener todo desde la tabla `Reservas` y todo de la tabla `Botes`. Revisa que agregaste todos los datos correctamente.

In [27]:
# Reserva
%sql SELECT * FROM Reservas

 * sqlite:///capitanes.db
Done.


cid,bid,fecha
23,101,2016-10-10
23,102,2016-10-10
23,103,2016-08-10
23,104,2017-07-10
31,102,2017-10-11
31,103,2018-11-06
31,104,2018-11-12
64,101,2018-09-05
64,102,2018-09-08
74,103,2018-09-08


In [28]:
# Botes
%sql SELECT * FROM Botes

 * sqlite:///capitanes.db
Done.


bid,bnombre,bcolor
101,Catamaran,Azul
102,Catamaran,Rojo
103,Endurance,Verde
104,Yate,Rojo


Ahora ejecuta la consulta a continuación. ¿Qué crees que retorna?

In [29]:
%sql SELECT * FROM Capitanes WHERE cnombre='Juan'

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
29,Juan,1.0,33


Ahora es tu turno, realiza las siguientes consultas en SQL:

**3.2.** Busca los nombres de los capitanes con rating más de 4 (prueba seleccionando `*` y luego solo el atributo `cnombre`).

**Ayuda**: Puedes usar los filtros `x = y`, `x < y`, `x <= y`, `<>` (distinto), etc. Para los números es obvio, para los strings es orden lexicográfico.

In [43]:
%%sql
SELECT cnombre
FROM Capitanes
WHERE crating > 4;


 * sqlite:///capitanes.db
Done.


cnombre
King Arturo
Andy
Felipe
Oscar
Isidora
Pedro
Isidora
Romano


**4.** Similarmente, busca los nombres de los capitanes con rating más de 4 y cuya edad está entre 25 y 40 años.

**Ayuda**: Combina condiciones en el `WHERE` usando `AND` y `OR`. Recuerda que para negar algo puedes usar `NOT`.

In [30]:
%%sql
SELECT cnombre
FROM Capitanes
WHERE crating > 4
AND cedad BETWEEN 25 AND 40;


 * sqlite:///capitanes.db
Done.


cnombre
King Arturo
Felipe
Oscar
Isidora
Isidora


**5.** Busca la edad de los capitanes que no tienen una `'a'` en su nombre (recuerda usar `LIKE`).

In [31]:
%%sql
SELECT cedad, cnombre
FROM Capitanes
WHERE cnombre NOT LIKE '%a%';

 * sqlite:///capitanes.db
Done.


cedad,cnombre
25,Felipe
16,Pedro


Escribe una consulta que entregue los nombres y rating de los capitanes, pero ordenado por `crating`. Para ordenar agrega un `ORDER BY (<atr_1>, ..., <atr_N>) DESC` al final de tu consulta. Eso ordena los resultados por el atributo `<atr_1>`, después `<atr_2>`, etc. Prueba que pasa al eliminar el `DESC` al final del `ORDER BY`.

### Consultas de _join_, unión, intersección y diferencia

Imaginemos que queremos encontrar los nombres de los capitanes que reservaron el bote 103.

**6.** Escribe primero esta consulta en Álgebra relacional.

Una forma directa para traspasarlo a SQL es con el comando `INNER JOIN`:

In [48]:
%%sql
SELECT  cnombre
FROM  Capitanes INNER JOIN Reservas
ON Capitanes.cid = Reservas.cid
WHERE Reservas.bid = 103

 * sqlite:///capitanes.db
Done.


cnombre
King Arturo
Andy
Isidora


Este es un buen minuto para repasar los joins. Puedes correr la consulta de arriba sin el `WHERE` para ver lo que sería el resultado de $\text{Capitanes} \bowtie_{\text{cid}=\text{cid}} \text{Reservas}$. ¿Qué pasa al agregar la condición del `WHERE`?
Si bien existe la instrucción `INNER JOIN`, una forma más clara (o que a nosotros nos gusta más) es explotando el producto cartesiano:

In [49]:
%%sql
SELECT  *
FROM  Capitanes, Reservas
WHERE Capitanes.cid = Reservas.cid
AND Reservas.bid = 103

 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad,cid_1,bid,fecha
23,King Arturo,8.0,31,23,103,2016-10-10
31,Andy,8.0,55,31,103,2018-11-06
74,Isidora,9.0,35,74,103,2018-09-08


En este caso estamos aplicando el `WHERE` al Producto Cartesiano, y por eso incluimos la condición `WHERE Capitanes.cid = Reservas.cid`. ¿Entiendes por qué ambas consultas son equivalentes?

Ahora intenta escribir las siguientes consultas:

**7.** Los nombres de los capitanes que reservaron un bote rojo.

In [32]:
%%sql
SELECT DISTINCT Capitanes.cnombre
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
JOIN Botes ON Reservas.bid = Botes.bid
WHERE Botes.bcolor = 'Rojo'

 * sqlite:///capitanes.db
Done.


cnombre
King Arturo
Andy
Isidora


**8.** Los nombres de los capitanes que reservaron un bote rojo o un bote verde.

In [34]:
%%sql
SELECT DISTINCT Capitanes.cnombre, botes.bcolor
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
JOIN Botes ON Reservas.bid = Botes.bid
WHERE Botes.bcolor = 'Rojo' OR Botes.bcolor = 'Verde';

 * sqlite:///capitanes.db
Done.


cnombre,bcolor
King Arturo,Rojo
King Arturo,Verde
Andy,Rojo
Andy,Verde
Isidora,Rojo
Isidora,Verde


**9.** La misma anterior, pero usando `UNION` para unir el resultado de dos consultas. Prueba usando `UNION` y luego `UNION ALL`. ¿Cuál es la diferencia?

In [35]:
%%sql
SELECT Capitanes.cnombre
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
JOIN Botes ON Reservas.bid = Botes.bid
WHERE Botes.bcolor = 'Rojo'

UNION

SELECT Capitanes.cnombre
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
JOIN Botes ON Reservas.bid = Botes.bid
WHERE Botes.bcolor = 'Verde';


 * sqlite:///capitanes.db
Done.


cnombre
Andy
Isidora
King Arturo


In [36]:
%%sql
SELECT Capitanes.cnombre
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
JOIN Botes ON Reservas.bid = Botes.bid
WHERE Botes.bcolor = 'Rojo'

UNION ALL

SELECT Capitanes.cnombre
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
JOIN Botes ON Reservas.bid = Botes.bid
WHERE Botes.bcolor = 'Verde';


 * sqlite:///capitanes.db
Done.


cnombre
King Arturo
King Arturo
Andy
Andy
Isidora
King Arturo
Andy
Isidora


**10.** Usa `AS` para cambiar de nombres a los atributos (esto es útil no solo para la unión!). La siguiente consulta entrega todos los ids del sistema:

In [ ]:
%%sql
SELECT cid AS id
FROM  Capitanes
UNION
SELECT bid AS id
FROM Botes

**11.** Los nombres de los capitanes que reservaron un bote rojo y un bote verde. Puedes usar `INTERSECT` para intersectar el resultado de dos consultas. ¿Puedes escribir también esta consulta usando AND?

In [37]:
%%sql
SELECT Capitanes.cnombre
FROM Capitanes
JOIN Reservas AS R1 ON Capitanes.cid = R1.cid
JOIN Reservas AS R2 ON Capitanes.cid = R2.cid
JOIN Botes AS B1 ON R1.bid = B1.bid
JOIN Botes AS B2 ON R2.bid = B2.bid
WHERE B1.bcolor = 'Rojo'
AND B2.bcolor = 'Verde';

 * sqlite:///capitanes.db
Done.


cnombre
King Arturo
King Arturo
Andy
Andy


### Sub-consultas y consultas anidadas

Has observado que cada consulta entrega una tabla, los atributos de esa tabla son lo que pones en el `SELECT`. Esto significa que podemos poner cualquier consulta SQL en el `FROM`. Vamos a ver cómo realizar esto.

**12.** Escribe una consulta que entregue los nombres de los capitanes junto a los bid de los botes que han reservado, pero contando solo las reservas hechas después del primero de octubre del 2017.

In [38]:
%%sql
SELECT Capitanes.cnombre, Reservas.bid
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
WHERE Reservas.fecha > '2017-10-01';


 * sqlite:///capitanes.db
Done.


cnombre,bid
Andy,102
Andy,103
Andy,104
Isidora,101
Isidora,102
Isidora,103


Sea $Q$ la consulta anterior. ¿Qué va a retornar la siguiente consulta? (Ojo, tienes que reemplazar el valor de $Q$ en la celda siguiente)

In [41]:
%%sql
SELECT Botes.bid, bnombre, bcolor
FROM Botes, (Reservas) AS Res2017
WHERE Botes.bid = Res2017.bid

 * sqlite:///capitanes.db
Done.


bid,bnombre,bcolor
101,Catamaran,Azul
102,Catamaran,Rojo
103,Endurance,Verde
104,Yate,Rojo
102,Catamaran,Rojo
103,Endurance,Verde
104,Yate,Rojo
101,Catamaran,Azul
102,Catamaran,Rojo
103,Endurance,Verde


**Importante**: cada vez que quieras anidar una consulta en el `FROM`, debes darle un alias. En el caso anterior el alias es Res2017.

**13.** Ahora modifica $Q$ para que solo retorne los bid de los botes, y no los nombres de los capitanes. ¿Qué retorna la consulta a continuación? ¿Qué pasa si Q entrega dos atributos en vez de uno (es decir, usando $Q$ sin modificar)? ¿Por qué pasa esto?

In [42]:
%%sql
SELECT Botes.bid, bnombre, bcolor
FROM Botes
WHERE Botes.bid IN Q

 * sqlite:///capitanes.db
(sqlite3.OperationalError) sub-select returns 3 columns - expected 1
[SQL: SELECT Botes.bid, bnombre, bcolor
FROM Botes
WHERE Botes.bid IN Reservas]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


También podemos usar consultas anidadas en el `WHERE`. Además del `IN` podemos usar `NOT IN`, o `> ANY`, `> ALL` para cualquier comparación entre `<`, `>`, `<=`, `>=`, `=`, y también anteponiendo `NOT`.

**Ojo**: `ANY` y `All` no funcionan en SQLite, pero es muy común en los otros sistemas.

Por ejemplo, si queremos encontrar los capitanes que no han reservado un bote verde:

1. Primero escribe una consulta que entregue los `cid` de los capitanes que han reservado un bote verde.
2. Sea $Q$ la consulta anterior. Los capitanes que no han reservado un bote verde se pueden extraer remplazando $Q$ en:

In [ ]:
%%sql
SELECT cnombre
FROM capitanes
WHERE cid NOT IN (Q)

Ahora:

**14.** Encuentre los nombres de los capitanes que tienen más de 20 años y que no han reservado un bote rojo. ¿Puedes escribir esto sin usar consultas anidadas?

In [43]:
%%sql
SELECT Capitanes.cnombre
FROM Capitanes
WHERE Capitanes.cedad > 20
AND NOT EXISTS (
    SELECT 1
    FROM Reservas
    JOIN Botes ON Reservas.bid = Botes.bid
    WHERE Reservas.cid = Capitanes.cid
    AND Botes.bcolor = 'Rojo'
);


 * sqlite:///capitanes.db
Done.


cnombre
Juan
Felipe
Oscar
Isidora
Rosa
Romano


**15.** Encuentre los nombres de los capitanes que reservaron al menos dos botes.

In [44]:
%%sql
SELECT Capitanes.cnombre
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
GROUP BY Capitanes.cid, Capitanes.cnombre
HAVING COUNT(Reservas.bid) >= 2;


 * sqlite:///capitanes.db
Done.


cnombre
King Arturo
Andy
Isidora


**16.** Encuentre los nombres de los capitanes que han reservado todos los botes. ¿Puedes escribir esto sin usar consultas anidadas?

In [45]:
%%sql
SELECT Capitanes.cnombre
FROM Capitanes
JOIN Reservas ON Capitanes.cid = Reservas.cid
GROUP BY Capitanes.cid, Capitanes.cnombre
HAVING COUNT(DISTINCT Reservas.bid) = (SELECT COUNT(*) FROM Botes);


 * sqlite:///capitanes.db
Done.


cnombre
King Arturo


### Agregación

¿Qué pasa si queremos el capitán con el mejor rating? Primero, podemos obtener el mejor rating con la siguiente consulta:

In [ ]:
%%sql
SELECT MAX(crating)
FROM Capitanes

Luego seleccionamos los nombres de aquellos capitanes con ese rating:

In [44]:
%%sql
SELECT cnombre
FROM Capitanes
WHERE crating = (SELECT MAX(crating) FROM Capitanes)

 * sqlite:///capitanes.db
Done.


cnombre
Oscar
Pedro


En general SQL soporta varias funciones de agregación: `MAX`, `MIN` (mínimo), `AVG` (promedio), `SUM` (suma). Intenta hacer las siguientes consultas:

**17.** Encuentra el rating promedio de los capitanes mayores a 45 años.

In [40]:
%%sql
SELECT AVG(crating)
FROM Capitanes
WHERE cedad > 45;

 * sqlite:///capitanes.db
Done.


AVG(crating)
6.75


**18.** Encuentra los capitanes cuya edad es menor que el promedio.

In [37]:
%%sql
SELECT *
FROM Capitanes
WHERE cedad < (SELECT AVG(cedad) FROM Capitanes);


 * sqlite:///capitanes.db
Done.


cid,cnombre,crating,cedad
23,King Arturo,8.0,31
29,Juan,1.0,33
32,Felipe,8.4,25
58,Oscar,10.0,35
64,Isidora,7.5,35
71,Pedro,10.0,16
74,Isidora,9.0,35
85,Rosa,3.0,25


**19.** Encuentra los botes que no han sido reservados por el capitán con peor rating.

In [45]:
%%sql
SELECT *
FROM Botes
WHERE bid NOT IN (
SELECT DISTINCT r.bid
FROM Reservas r
JOIN Capitanes c ON r.cid = c.cid
WHERE c.crating = (SELECT MIN(crating) FROM Capitanes));


 * sqlite:///capitanes.db
Done.


bid,bnombre,bcolor
101,Catamaran,Azul
102,Catamaran,Rojo
103,Endurance,Verde
104,Yate,Rojo


También podemos usar `COUNT` para contar tuplas. Hay dos versiones (pruébalas!):

- `SELECT COUNT(*) FROM Tabla` cuenta cuantas tuplas tiene la tabla.
- `SELECT COUNT(DISTINCT cnombre) FROM Capitanes` cuenta cuantos nombres distintos hay en Capitanes.

En general `SELECT COUNT(*) FROM Tabla` y S`ELECT COUNT(atributo) FROM Tabla` es lo mismo, pero hay casos en los que es distinto: cuando las tablas tienen valores nulos o indefinidos. Hablaremos sobre esto más adelante en el curso.

Ahora realiza las siguientes consultas.

**20.** Encuentra cuantos capitanes menores de 40 años hay.

In [47]:
%%sql
SELECT COUNT(*) AS Cap_menosres_de_40
FROM Capitanes
WHERE cedad < 40;

 * sqlite:///capitanes.db
Done.


Cap_menosres_de_40
8


**21.** Cuenta la cantidad de reservas hechas a botes de color rojo.

In [48]:
%%sql
SELECT COUNT(*) AS Cant_de_Reservas
FROM Reservas r
JOIN Botes b ON r.bid = b.bid
WHERE b.bcolor = 'Rojo';


 * sqlite:///capitanes.db
Done.


Cant_de_Reservas
5


**22.** Cuenta cuantos capitanes tienen rating mayor al promedio y luego cuantos capitanes tienen rating menor al promedio.

In [49]:
%%sql
SELECT COUNT(*) AS Cap_mayor_promedio
FROM Capitanes
WHERE crating > (SELECT AVG(crating) FROM Capitanes);

 * sqlite:///capitanes.db
Done.


Cap_mayor_promedio
7


Finalmente podemos usar `GROUP BY` para ir agrupando los resultados. Prueba con la consulta:

In [ ]:
%%sql
SELECT cnombre, SUM(cedad)
FROM Capitanes
GROUP BY cnombre

¿Qué pasó con Isidora? Lo que hacemos es agrupar a las dos tuplas con nombre Isidora en una, y sumamos la edad. Prueba ahora:

In [ ]:
%%sql
SELECT bcolor, count(bcolor)
FROM Botes
GROUP BY bcolor

Ahora realiza las siguientes consultas:

**23.** Encuentra para los capitanes el promedio del ranking por cada edad distinta.

In [51]:
%%sql
SELECT cedad AS Edad_capitan,AVG(crating) AS Prom_del_ranking
FROM Capitanes
GROUP BY cedad;

 * sqlite:///capitanes.db
Done.


Edad_capitan,Prom_del_ranking
16,10.0
25,5.7
31,8.0
33,1.0
35,8.833333333333334
55,8.0
63,5.5


**24.** Encuentra el número de reservas por cada color de bote.

In [52]:
%%sql
SELECT bcolor AS Color_bote,COUNT(*) AS Numero_reservas
FROM Reservas r
JOIN Botes b ON r.bid = b.bid
GROUP BY bcolor;


 * sqlite:///capitanes.db
Done.


Color_bote,Numero_reservas
Azul,2
Rojo,5
Verde,3


**25.** Encuentra el numero de reservas por capitán y por color de bote.

In [53]:
%%sql
SELECT c.cnombre AS Nombre_capitan, b.bcolor AS Color_bote, COUNT(r.bid) AS Numero_reservas
FROM Capitanes c
JOIN Reservas r ON c.cid = r.cid
JOIN Botes b ON r.bid = b.bid
GROUP BY c.cnombre, b.bcolor;


 * sqlite:///capitanes.db
Done.


Nombre_capitan,Color_bote,Numero_reservas
Andy,Rojo,2
Andy,Verde,1
Isidora,Azul,1
Isidora,Rojo,1
Isidora,Verde,1
King Arturo,Azul,1
King Arturo,Rojo,2
King Arturo,Verde,1


### Inserción y eliminación más compleja

Lo último que veremos es una forma de insertar y eliminar tuplas de forma más compleja. Por ejemplo podemos insertar valores de una consulta.

**26.** Crea una tabla `CapitanesTop(cid INT, cnombre VARCHAR(100))`

In [42]:
%%sql
CREATE TABLE CapitanesTop (cid INT,cnombre VARCHAR(100));


 * sqlite:///capitanes.db
Done.


[]

Ahora ejecuta la siguiente inserción. Nota que para insertar desde una consulta no debemos escribir la instrucción `VALUES`.

In [ ]:
%%sql
INSERT INTO CapitanesTop
(SELECT cid,cnombre
FROM Capitanes
WHERE Rating > 8)

Finalmente, si quieres dejar capitanes que tienen un rating igual o superior a 9 puedes escribir la siguiente consulta:

In [ ]:
%%sql
DELETE FROM CapitanesTop
WHERE cid IN
  (SELECT cid
  FROM Capitanes
  WHERE crating < 9)